---
sidebar_label: FireCrawl

---

# FireCrawlLoader

本笔记本简要介绍了如何快速开始使用 [FireCrawlLoader](/docs/integrations/document_loaders/)。如需了解 FireCrawlLoader 所有功能和配置的详细文档，请前往 [API 参考](https://api.js.langchain.com/classes/langchain_community_document_loaders_web_firecrawl.FireCrawlLoader.html)。

## 概览
### 集成详情

| 类 | 包 | 本地支持 | 可序列化 | [Python 支持](https://python.langchain.com/docs/integrations/document_loaders/firecrawl)|
| :--- | :--- | :---: | :---: |  :---: |
| [FireCrawlLoader](https://api.js.langchain.com/classes/langchain_community_document_loaders_web_firecrawl.FireCrawlLoader.html) | [@langchain/community](https://api.js.langchain.com/modules/langchain_community_document_loaders_web_firecrawl.html) | 🟠（详见下文） | beta | ✅ | 
### 加载器特性
| 来源 | 网络加载器 | 仅限 Node 环境
| :---: | :---: | :---: | 
| FireCrawlLoader | ✅ | ❌ | 

[FireCrawl](https://firecrawl.dev) 可以爬取并将任何网站转换为适用于 LLM 的数据。它会爬取所有可访问的子页面，并为每个页面提供干净的 Markdown 和元数据。无需站点地图。

FireCrawl 能够处理诸如反向代理、缓存、速率限制以及 JavaScript 阻挡的内容等复杂任务。由 [mendable.ai](https://mendable.ai) 团队开发。

本指南展示了如何抓取整个网站并使用 LangChain 中的 `FireCrawlLoader` 加载它们。

## 准备工作

要使用 `FireCrawlLoader` 文档加载器，你需要安装 `@langchain/community` 集成和 `@mendable/firecrawl-js@0.0.36` 包。然后创建一个 **[FireCrawl](https://firecrawl.dev)** 账号并获取 API 密钥。

### 凭证信息

注册并获取你的免费 [FireCrawl API 密钥](https://firecrawl.dev)。FireCrawl 提供了 300 个免费积分供你开始使用，如果你想自建，它也是 [开源的](https://github.com/mendableai/firecrawl)。

完成注册后，请设置 `FIRECRAWL_API_KEY` 环境变量：

```bash
export FIRECRAWL_API_KEY="your-api-key"
```

如果你想获得模型调用的自动追踪，也可以取消下面代码的注释并设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain 的 FireCrawlLoader 集成位于 `@langchain/community` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core @mendable/firecrawl-js@0.0.36
</Npm2Yarn>

```

## 实例化

以下是如何使用 `FireCrawlLoader` 加载网页搜索结果的示例：

Firecrawl 提供 3 种模式：`scrape`、`crawl` 和 `map`。在 `scrape` 模式下，Firecrawl 仅会抓取你提供的页面。在 `crawl` 模式下，Firecrawl 会爬取整个网站。在 `map` 模式下，Firecrawl 将返回与网站相关的语义链接。

参数 `formats`（`crawl` 模式下为 `scrapeOptions.formats`）允许从 `"markdown"`、`"html"` 或 `"rawHtml"` 中进行选择。不过，加载的文档将仅以一种格式返回内容，优先级如下：`markdown`、然后是 `html`、最后是 `rawHtml`。

现在我们可以实例化模型对象并加载文档：

In [ ]:
import "@mendable/firecrawl-js";
import { FireCrawlLoader } from "@langchain/community/document_loaders/web/firecrawl"

const loader = new FireCrawlLoader({
  url: "https://firecrawl.dev", // The URL to scrape
  apiKey: "...", // Optional, defaults to `FIRECRAWL_API_KEY` in your env.
  mode: "scrape", // The mode to run the crawler in. Can be "scrape" for single urls or "crawl" for all accessible subpages
  params: {
    // optional parameters based on Firecrawl API docs
    // For API documentation, visit https://docs.firecrawl.dev
  },
})

## 加载

In [3]:
const docs = await loader.load()
docs[0]

Document {
  pageContent: "Introducing [Smart Crawl!](https://www.firecrawl.dev/smart-crawl)\n" +
    " Join the waitlist to turn any web"... 18721 more characters,
  metadata: {
    title: "Home - Firecrawl",
    description: "Firecrawl crawls and converts any website into clean markdown.",
    keywords: "Firecrawl,Markdown,Data,Mendable,Langchain",
    robots: "follow, index",
    ogTitle: "Firecrawl",
    ogDescription: "Turn any website into LLM-ready data.",
    ogUrl: "https://www.firecrawl.dev/",
    ogImage: "https://www.firecrawl.dev/og.png?123",
    ogLocaleAlternate: [],
    ogSiteName: "Firecrawl",
    sourceURL: "https://firecrawl.dev",
    pageStatusCode: 500
  },
  id: undefined
}

In [4]:
console.log(docs[0].metadata)

{
  title: "Home - Firecrawl",
  description: "Firecrawl crawls and converts any website into clean markdown.",
  keywords: "Firecrawl,Markdown,Data,Mendable,Langchain",
  robots: "follow, index",
  ogTitle: "Firecrawl",
  ogDescription: "Turn any website into LLM-ready data.",
  ogUrl: "https://www.firecrawl.dev/",
  ogImage: "https://www.firecrawl.dev/og.png?123",
  ogLocaleAlternate: [],
  ogSiteName: "Firecrawl",
  sourceURL: "https://firecrawl.dev",
  pageStatusCode: 500
}


## 附加参数

对于 `params`，您可以根据 [Firecrawl 文档](https://docs.firecrawl.dev) 中的参数进行传递。

## API 参考文档

如需详细了解 FireCrawlLoader 的所有功能和配置，请访问 API 参考文档：https://api.js.langchain.com/classes/langchain_community_document_loaders_web_firecrawl.FireCrawlLoader.html